In [4]:
import utils.feature_extractors as utils
from utils.evaluation import action_evaluator
import numpy as np
import os
import json
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from itertools import product
from sklearn.svm import SVC
import optuna
import joblib

d:\Academic_PC\Sem 8\DruggableProtienPrediction\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = {
    "train_pos_loc":"data/TR_pos_SPIDER.txt",
    "train_neg_loc":"data/TR_neg_SPIDER.txt",
    "test_pos_loc":"data/TS_pos_SPIDER.txt",
    "test_neg_loc":"data/TS_neg_SPIDER.txt",
    "model_save_loc":"optimized",
    "artifact_dir":"artifacts",
    "random_seed":9
}

In [3]:
sel_model = {
    "AAC":"SVC",
    "DPC":"SVC",
    "CTD":"LGBMClassifier",
    "PAAC":"RandomForest",
    "APAAC":"SVC",
    "RSacid":"SVC",
    "RSpolar":"SVC",
    "RSsecond":"SVC",
    "RScharge":"SVC",
    "RSDHP":"SVC",
    "Combine":"LGBMClassifier",
}

## AAC

In [8]:
feat_type = "AAC"
study = joblib.load(os.path.join(config["model_save_loc"],feat_type,sel_model[feat_type],"study.sav"))
    
with open(os.path.join(config["model_save_loc"],feat_type,sel_model[feat_type],"best_params.json"),"r") as f0:
    params = json.load(f0)
 

optuna.visualization.plot_slice(study)

## CTD

In [9]:
feat_type = "CTD"
study = joblib.load(os.path.join(config["model_save_loc"],feat_type,sel_model[feat_type],"study.sav"))
    
with open(os.path.join(config["model_save_loc"],feat_type,sel_model[feat_type],"best_params.json"),"r") as f0:
    params = json.load(f0)
 

optuna.visualization.plot_slice(study)

In [10]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])

## PAAC

In [11]:
feat_type = "PAAC"
study = joblib.load(os.path.join(config["model_save_loc"],feat_type,sel_model[feat_type],"study.sav"))
    
with open(os.path.join(config["model_save_loc"],feat_type,sel_model[feat_type],"best_params.json"),"r") as f0:
    params = json.load(f0)
 

optuna.visualization.plot_slice(study)

In [12]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])